# Import Lib

In [18]:
# Import các thư viện cần thiết
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# Thiet lap Login va crawl data

In [19]:
# Đường dẫn đến ChromeDriver
chrome_driver_path = 'path/to/chromedriver.exe'
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()

# Khởi tạo trình duyệt
driver = webdriver.Chrome(service=service, options=options)

# Mở trang Facebook và chờ người dùng đăng nhập thủ công
driver.get('https://www.facebook.com')
time.sleep(3)  # Đợi trang tải
username = driver.find_element(By.ID, 'email')
password = driver.find_element(By.ID, 'pass')

# Điền thông tin đăng nhập (thay bằng tài khoản của bạn)
username.send_keys('email_to_login')
password.send_keys('password_to_login')

# Đăng nhập
login_button = driver.find_element(By.NAME, 'login')
login_button.click()
time.sleep(5)  # Đợi trang đăng nhập hoàn tất

# Điều hướng tới group
group_url = 'https://www.facebook.com/groups/tuyensinhddq'
driver.get(group_url)
time.sleep(5)  # Đợi trang tải

# Số bài đăng muốn lấy
desired_post_count = 1000
collected_posts = 0  # Khởi tạo bộ đếm bài đăng
data = []  # Khởi tạo danh sách chứa dữ liệu bài đăng

# Loop cho đến khi thu thập đủ số bài đăng
while collected_posts < desired_post_count:
    # Lấy tất cả các bài đăng trong group (dựa trên class đã cung cấp)
    posts = driver.find_elements(By.CSS_SELECTOR, 'div[role="feed"] div.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
    
    # Kiểm tra số lượng bài đăng đã thu thập được
    for post in posts:
        post_data = {}
        
        # Lấy nội dung bài đăng
        try:
            content_element = post.find_element(By.CSS_SELECTOR, 'div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs.x126k92a')
            post_data['PostContent'] = content_element.text
        except:
            post_data['PostContent'] = None

        # Lấy ngày đăng bài từ thuộc tính aria-label của thẻ a
        try:
            date_element = post.find_element(By.CSS_SELECTOR, 'span.x4k7w5x.x1h91t0o.x1h9r5lt.x1jfb8zj.xv2umb2.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1qrby5j a')
            post_data['Date'] = date_element.get_attribute('aria-label')
        except:
            post_data['Date'] = None

        # Lấy số bình luận
        try:
            comment_count_element = post.find_element(By.CSS_SELECTOR, 'span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs.x1sur9pj.xkrqix3')
            post_data['NumberOfComment'] = comment_count_element.text
        except:
            post_data['NumberOfComment'] = None

        # Lấy số like, heart, care, haha, wow, sad, angry
        try:
            react_element = post.find_elements(By.CSS_SELECTOR, "span[class='x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen x1s688f xi81zsa']")
            
            react_text = [react.text for react in react_element]
            # Chỉ giữ lại các reaction không phải "Tất cả"
            react_text = [text for text in react_text if text.strip() != "Tất cả"]
 
            # Mặc định tất cả là 0, sẽ cập nhật nếu tìm thấy
            reactions = {'Like': 0, 'Heart': 0, 'Care': 0, 'Haha': 0, 'Wow': 0, 'Sad': 0, 'Angry': 0}

            # Phân tích số lượng reaction
            for react in react_text:
                parts = react.split(' ')
                if 'Thịch' in react:
                    reactions['Like'] = parts[0]
                elif 'Yêu thích' in react:
                    reactions['Heart'] = parts[0]
                elif 'Thương thương' in react:
                    reactions['Care'] = parts[0]
                elif 'Haha' in react:
                    reactions['Haha'] = parts[0]
                elif 'Wow' in react:
                    reactions['Wow'] = parts[0]
                elif 'Buồn' in react:
                    reactions['Sad'] = parts[0]
                elif 'Phẩn nộ' in react:
                    reactions['Angry'] = parts[0]

            post_data.update(reactions)
        except:
            post_data.update({'Like': 0, 'Heart': 0, 'Care': 0, 'Haha': 0, 'Wow': 0, 'Sad': 0, 'Angry': 0})

        # Lấy tất cả các bình luận của bài đăng
        try:
            comment_elements = post.find_elements(By.CSS_SELECTOR, 'div.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs')
            comments = [comment.text for comment in comment_elements]
            post_data['ContentofComment'] = comments
        except:
            post_data['ContentofComment'] = []

        
        # Kiểm tra nếu bài đăng có đủ dữ liệu
        if post_data['PostContent'] and post_data['Date'] and post_data['NumberOfComment']:
            # Thêm dữ liệu bài đăng vào danh sách nếu đủ thông tin
            data.append(post_data)
            print("collected_post: ", collected_posts)
            
            # Tăng bộ đếm bài đăng
            collected_posts += 1
        
        # Kiểm tra nếu đã thu thập đủ bài đăng
        if collected_posts >= desired_post_count:
            break

    # Scroll thêm nếu chưa thu thập đủ bài đăng
    if collected_posts < desired_post_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Đợi trang tải thêm nội dung

# Đóng trình duyệt
driver.quit()


collected_post:  0
collected_post:  1
collected_post:  2
collected_post:  3
collected_post:  4
collected_post:  5
collected_post:  6
collected_post:  7
collected_post:  8
collected_post:  9
collected_post:  10
collected_post:  11
collected_post:  12
collected_post:  13
collected_post:  14
collected_post:  15
collected_post:  16
collected_post:  17
collected_post:  18
collected_post:  19
collected_post:  20
collected_post:  21
collected_post:  22
collected_post:  23
collected_post:  24
collected_post:  25
collected_post:  26
collected_post:  27
collected_post:  28
collected_post:  29
collected_post:  30
collected_post:  31
collected_post:  32
collected_post:  33
collected_post:  34
collected_post:  35
collected_post:  36
collected_post:  37
collected_post:  38
collected_post:  39
collected_post:  40
collected_post:  41
collected_post:  42
collected_post:  43
collected_post:  44
collected_post:  45
collected_post:  46
collected_post:  47
collected_post:  48
collected_post:  49
collected_

# Export data

In [20]:
# Lưu dữ liệu vào file CSV
df = pd.DataFrame(data)

# Loại bỏ những dòng dữ liệu trùng lặp dựa trên nội dung bài đăng, ngày đăng và số bình luận
df = df.drop_duplicates(subset=['PostContent', 'Date', 'NumberOfComment'], keep='first')

df.to_csv('facebook_group_data.csv', index=False)

# Xem trước dữ liệu
df.head()

,PostContent,Date,NumberOfComment,Like,Heart,Care,Haha,Wow,Sad,Angry,ContentofComment
0,CHÍNH THỨC MỞ ĐƠN TUYỂN THÀNH VIÊN DUET - NHÓ...,32 phút,2,0,0,0,0,0,0,0,[ CHÍNH THỨC MỞ ĐƠN TUYỂN THÀNH VIÊN DUET - NH...
2,Cho e hỏi trường mình có trao đổi sv với các t...,4 ngày,1,0,0,0,0,0,0,0,[Cho e hỏi trường mình có trao đổi sv với các ...
3,Nhanh tay apply để tham gia vào hành trình săn...,5 Tháng 10 lúc 20:19,20,0,0,0,0,0,0,0,[Nhanh tay apply để tham gia vào hành trình să...
4,[TUYỂN THÀNH VIÊN DỰ ÁN GREEN UNIVERSITY DUE -...,5 Tháng 10 lúc 20:19,3,0,0,0,0,0,0,0,[[TUYỂN THÀNH VIÊN DỰ ÁN GREEN UNIVERSITY DUE ...
5,Cho em hỏi đăng kí thi bằng lái xe ở trường th...,5 Tháng 10 lúc 07:07,5,0,0,0,0,0,0,0,[Cho em hỏi đăng kí thi bằng lái xe ở trường t...
